Да, вы совершенно правы. При малом количестве данных (5-10 значений) среднее арифметическое и медиана могут быть очень чувствительны к выбросам и не всегда надежно отражают "типичное" значение. К счастью, есть несколько методов, которые можно использовать для повышения устойчивости при расчете среднего и медианы, особенно в ситуациях, когда данные могут содержать выбросы или когда их мало.

Вот несколько подходов и их объяснение:

1. Тримированное среднее (Trimmed Mean)

•  Идея: Исключить определенный процент самых маленьких и самых больших значений перед расчетом среднего. Это помогает избавиться от влияния выбросов.
•  Как это работает: Вы выбираете процент (например, 10% или 20%), который будет отброшен с обеих сторон распределения.
•  Преимущества: Более устойчиво к выбросам, чем обычное среднее.
•  Недостатки: Требует выбора процента для обрезки, что может быть немного произвольно.

2. Усеченное среднее (Winsorized Mean)

•  Идея: Вместо удаления крайних значений, заменяет их на ближайшие оставшиеся.
•  Как это работает: Вы выбираете процент (например, 10% или 20%), который будет "усечен" с каждой стороны распределения. Значения за пределами этих границ заменяются на ближайшее "не усеченное" значение.
•  Преимущества: Более устойчиво к выбросам, чем обычное среднее, использует все данные, а не отбрасывает их.
•  Недостатки: Требует выбора процента усечения, что также может быть произвольным.

3. M-оценки (Robust Estimators)

•  Идея: Использовать более сложную функцию влияния, которая уменьшает вес выбросов при расчете среднего.
•  Как это работает: Вместо того, чтобы присваивать всем значениям одинаковый вес (как в обычном среднем), M-оценки используют весовую функцию, которая уменьшает вес значений, далеких от центра распределения. Примеры:
  •  Huber M-оценка: Компромисс между устойчивостью и эффективностью.
  •  Tukey's biweight (bisquare): Более устойчива к выбросам, чем Huber.
•  Преимущества: Очень устойчивы к выбросам.
•  Недостатки: Более сложные в расчете, чем простые средние и медианы.

4. Медиана с минимальной дисперсией (Median of Means)
• Идея: Случайным образом разбиваем данные на подгруппы. Находим среднее каждой подгруппы, затем вычисляем медиану получившихся средних.
• Как это работает: Выбираем размер подгруппы (например, 3), разбиваем случайным образом исходные данные на подгруппы, в которых вычисляем среднее. Из полученного набора средних вычисляем медиану.
• Преимущества: более устойчива чем среднее, и может быть более устойчивой чем медиана. Помогает снизить дисперсию.
• Недостатки: может быть сложнее в реализации

5. Бутстрап для медианы и среднего
•  Идея: Вместо одного среднего или медианы, мы проводим бутстрап и получаем распределение возможных значений среднего/медианы, что позволяет учесть неопределенность из-за малого количества данных.
•  Как это работает:
  •  Многократно (например, 1000 раз) создаем новые выборки того же размера, что и исходная выборка, но с заменой (то есть элементы могут повторяться).
  •  Для каждой новой выборки вычисляем среднее или медиану.
  •  Из полученного распределения средних или медиан можно оценить среднее и медиану с учетом их неопределенности.
•  Преимущества: Дает понимание, насколько сильно варьируется среднее/медиана при разных выборках.
•  Недостатки: Требует дополнительных вычислений.

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import trim_mean, tmean
from statsmodels.robust import scale
from statistics import median

def trimmed_mean(data, trim_percent=0.1):
    """Вычисляет тримированное среднее."""
    return trim_mean(data, trim_percent)

def winsorized_mean(data, trim_percent=0.1):
    """Вычисляет усеченное среднее."""
    
    if len(data) < 3:
       return np.mean(data)
    
    low_limit = np.quantile(data, trim_percent)
    high_limit = np.quantile(data, 1 - trim_percent)
    
    winsorized_data = np.clip(data, low_limit, high_limit)
    
    return np.mean(winsorized_data)


def huber_m_estimator(data, scale_est=None):
    """Вычисляет M-оценку Хубера."""
    if scale_est is None:
        # Using Median Absolute Deviation for scale estimation
        scale_est = scale.mad(data)
    else:
      scale_est = scale_est(data)
      
    if scale_est == 0:
      return np.mean(data)

    # Using Huber's loss function
    weights = np.ones_like(data, dtype=float)
    
    if scale_est!=0:
       weights = 1 / (1 + np.abs((data - np.mean(data)) / scale_est))
       
    return np.sum(weights * data) / np.sum(weights)

def tukey_m_estimator(data, scale_est=None):
    """Вычисляет M-оценку Тьюки."""
    if scale_est is None:
        # Using Median Absolute Deviation for scale estimation
        scale_est = scale.mad(data)
    else:
      scale_est = scale_est(data)

    if scale_est == 0:
      return np.mean(data)
    
    weights = np.zeros_like(data, dtype=float)
    u = np.abs((data - np.mean(data)) / scale_est)
    
    mask = u <= 1
    weights[mask] = (1-u[mask]**2)**2

    return np.sum(weights * data) / np.sum(weights)

def median_of_means(data, subsample_size):
    """Вычисляет медиану средних."""
    if len(data) < subsample_size:
       return np.mean(data)

    num_subsamples = len(data) // subsample_size
    subsamples = np.split(data[:num_subsamples * subsample_size], num_subsamples)
    
    means = [np.mean(subsample) for subsample in subsamples]
    
    return median(means)


def bootstrap_mean_median(data, num_iterations=1000):
   """Выполняет бутстрап для оценки среднего и медианы."""
   means = []
   medians = []
   for _ in range(num_iterations):
       sample = np.random.choice(data, size=len(data), replace=True)
       means.append(np.mean(sample))
       medians.append(median(sample))
   return np.mean(means), median(means), np.mean(medians), median(medians)


if __name__ == '__main__':
    # Пример использования
    data = np.array([10, 12, 15, 13, 120]) # Выброс
    # data = np.array([10, 12, 15, 13, 12]) #Без выброса
    
    print(f"Original data: {data}")
    print(f"Mean: {np.mean(data):.2f}")
    print(f"Median: {np.median(data):.2f}")
    print(f"Trimmed mean (10%): {trimmed_mean(data, trim_percent=0.1):.2f}")
    print(f"Winsorized mean (10%): {winsorized_mean(data, trim_percent=0.1):.2f}")
    print(f"Huber M-estimator (mad): {huber_m_estimator(data):.2f}")
    print(f"Tukey M-estimator (mad): {tukey_m_estimator(data):.2f}")
    print(f"Median of Means (subsample=2): {median_of_means(data, subsample_size=2):.2f}")
    mean_boot, median_mean_boot, median_boot, median_median_boot = bootstrap_mean_median(data)
    print(f"Bootstrap mean (mean of means): {mean_boot:.2f}")
    print(f"Bootstrap median (mean of medians): {median_mean_boot:.2f}")
    print(f"Bootstrap mean (median of medians): {median_boot:.2f}")
    print(f"Bootstrap median (median of medians): {median_median_boot:.2f}")

Как использовать код:

1. Скопируйте код в файл Python (например, robust_stats.py).
2. Замените пример данных на ваши данные.
3. Запустите скрипт python robust_stats.py.
4. Результаты будут выведены в консоль.

Рекомендации:

•  Выбор метода:
  •  Если у вас есть подозрение на выбросы, но их не очень много, попробуйте тримированное среднее или усеченное среднее с небольшим процентом отбрасывания/усечения (например, 10-20%).
  •  Если выбросов может быть больше и вы хотите более устойчивую оценку, попробуйте M-оценки Хубера или Тьюки.
  •  Если у вас мало данных, но при этом нет явных выбросов, то можно использовать медиану.
  •  Если хотите оценить неопределенность среднего или медианы при малом количестве данных, можно воспользоваться бутстрапом.
•  Визуализация: Всегда полезно визуализировать ваши данные (например, построить гистограмму или box plot), чтобы лучше понимать их распределение и наличие выбросов.
•  Экспериментируйте: Попробуйте разные методы и параметры, чтобы найти тот, который лучше всего подходит для ваших данных.

In [ ]:
def combined_robust_estimate(data):
    """Вычисляет комбинированную оценку, усредняя результаты различных методов."""

    estimates = []
    
    estimates.append(trimmed_mean(data, trim_percent=0.1))
    estimates.append(winsorized_mean(data, trim_percent=0.1))
    estimates.append(huber_m_estimator(data))
    estimates.append(tukey_m_estimator(data))
    estimates.append(median_of_means(data, subsample_size=2))
    mean_boot, median_mean_boot, median_boot, median_median_boot = bootstrap_mean_median(data)
    estimates.append(mean_boot)
    estimates.append(median_mean_boot)

    return np.mean(estimates), np.median(estimates)

# Пример использования
data = np.array([10, 12, 15, 13, 120]) # С выбросом
#data = np.array([10, 12, 15, 13, 12]) # Без выброса

print(f"Original data: {data}")
mean_combined, median_combined = combined_robust_estimate(data)
print(f"Combined Robust Estimate (mean of estimates): {mean_combined:.2f}")
print(f"Combined Robust Estimate (median of estimates): {median_combined:.2f}")